In [54]:
import torch
from datasets import Dataset, concatenate_datasets, load_from_disk
import os
import json
from datasets import (
    Dataset,
    DatasetDict,
    Features,
    Sequence,
    Value
)

In [55]:
rank = torch.load('/opt/ml/input/code/colbert/inferecne_colbert_rank.pth')

In [56]:
test_dataset = load_from_disk('/opt/ml/input/data/test_dataset')

In [57]:
with open(os.path.join("/opt/ml/input/data/wikipedia_documents.json"), "r", encoding="utf-8") as f:
    wiki = json.load(f)

contexts = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [58]:
df = test_dataset['validation'].to_pandas()

In [59]:
context = ''
tt = []
for row in rank:
    for i in row[:5]:
        context += contexts[i]
    tt.append(context)
    context = ''

In [60]:
df['context'] = tt

In [61]:
f = Features(
    {
        "context": Value(dtype="string", id=None),
        "id": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
    }
)


datasets = DatasetDict({"validation": Dataset.from_pandas(df, features=f)})

In [62]:
datasets.save_to_disk("/opt/ml/input/data/colbert_result")

In [63]:
datasets = load_from_disk("/opt/ml/input/data/colbert_result")